**versión con botones anterior-siguiente**

In [5]:
import dash
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objs as go

# Load Data
cap = pd.read_csv('capitales.csv')
# Build App
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("Recorrido sobre las capitales de Argentina"),
    
    html.Button('Siguiente', id='btn-siguiente', n_clicks=0),
    html.Button('Anterior', id='btn-anterior', n_clicks=0),
    html.Div(id='container-button-basic',
             children='Enter a value and press submit'),
    dcc.Graph(id='graph'),
])
# Define callback to update graph
@app.callback(
    [Output('graph', 'figure'),
     dash.dependencies.Output('container-button-basic', 'children')],
    [Input("btn-siguiente", 'n_clicks'),
     Input("btn-anterior", 'n_clicks')],
)

def update_figure(n_sig, n_ant):
    changed_id = [p['prop_id'] for p in dash.callback_context.triggered][0]
    pos_actual = n_sig - n_ant
    if 'btn-siguiente.n_clicks' in changed_id:
        fig = px.scatter_mapbox(cap, lat="latitud", lon="longitud", hover_name="capital", hover_data=["capital"],
                        color_discrete_sequence=["red"], zoom=3, height=500, width=500)
        fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
        fig.update_layout(mapbox_style="open-street-map")
        fig.add_trace(
            go.Scattermapbox(
                mode = "lines",
                lon = cap['longitud'][:pos_actual + 1],
                lat = cap['latitud'][:pos_actual + 1],
            )
        )
        return fig, f'pos_actual = {pos_actual} -- n_sig {n_sig}, n_ant {n_ant} -- changed_id {changed_id}'
        
    elif 'btn-anterior.n_clicks' in changed_id:  
        if pos_actual > len(cap):
            raise dash.exceptions.PreventUpdate
        else:    
            fig = px.scatter_mapbox(cap, lat="latitud", lon="longitud", hover_name="capital", hover_data=["capital"],
                            color_discrete_sequence=["red"], zoom=3, height=500, width=500)
            fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
            fig.update_layout(mapbox_style="open-street-map")
            fig.add_trace(
                go.Scattermapbox(
                    mode = "lines",
                    lon = cap['longitud'][:pos_actual + 1],
                    lat = cap['latitud'][:pos_actual + 1],
                )
            )
            return fig, f'pos_actual = {pos_actual} -- n_sig {n_sig}, n_ant {n_ant} -- changed_id {changed_id}'
    else:
        fig = px.scatter_mapbox(cap, lat="latitud", lon="longitud", hover_name="capital", hover_data=["capital"],
                            color_discrete_sequence=["red"], zoom=3, height=500, width=500)
        fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
        fig.update_layout(mapbox_style="open-street-map")
        return fig, f'pos_actual = {pos_actual} -- n_sig {n_sig}, n_ant {n_ant} -- changed_id {changed_id}'

# Run app and display result inline in the notebook
app.run_server(mode='jupyterlab', port=8080)

**Versión con slider**

In [2]:
import dash
import pandas as pd
import plotly.express as px
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output

# Load Data
cap = pd.read_csv('capitales.csv')

app = JupyterDash(__name__,
external_stylesheets=[dbc.themes.BOOTSTRAP])

# Build App
app.layout = html.Div([
    dbc.Row(dbc.Col(html.H3("Recorrido sobre las capitales de Argentina")
               )
       ),
    dbc.Row([dbc.Col(
            dcc.Slider(
                id='slider',
                vertical=True,
                min=1,
                max=23,
                marks={i: str(cap['capital'][i]) for i in range(1, len(cap))},
                value=23,
                verticalHeight=800
                )
            ),
            dbc.Col(dcc.Graph(id='graph'))
            ])
    
])
# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    Input('slider', 'value'),
)

def update_figure(slider_value):
    pos_actual = slider_value
    fig = px.scatter_mapbox(cap, lat="latitud", lon="longitud", hover_name="capital", hover_data=["capital"],
                    color_discrete_sequence=["red"], zoom=3, height=400, width=500)
    
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.update_layout(mapbox_style="open-street-map")
    
    fig.add_trace(
        go.Scattermapbox(
            mode = "lines",
            lon = cap['longitud'][:pos_actual],
            lat = cap['latitud'][:pos_actual],
        )
    )
    return fig

# Run app and display result inline in the notebook
app.run_server(mode='jupyterlab', port=8080)